In [147]:
import pandas as pd
import numpy as np

1. HTML = "html_analysis.py" results
2. logReport = "filter_html_download.py" results
3. inFile = 'startup_url_list.csv

In [156]:
html = pd.read_csv('../data/07_27_analysis.csv')
logReport = pd.read_csv('../data/HTMLReport.csv')
inFile = pd.read_csv('../data/startup_url_list.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(Temporary) delete: `Unnamed: 0` from `logReport`

In [157]:
logReport = logReport.drop('Unnamed: 0', axis=1)
logReport['capture_yr'] = logReport.timestamp.astype(str).str.slice(start=0, stop=4).astype(int)
logReport['capture_m'] = logReport.timestamp.astype(str).str.slice(start=4, stop=6).astype(int)

First, merge HTML with the inFile on entityid, and with the logReport on entityid, capture year, and capture month

In [158]:
html = pd.merge(html, logReport, on=['entityid', 'capture_yr', 'capture_m'])
html = html.merge(inFile, on='entityid')

Define companies as failed, not-failed (exited), or unknown

In [168]:
failedStatus = ['Out of Business', 'In Bankruptcy']
exitedStatus = ['Publicly-held', 'Acquired/Merged', 'In IPO Registration']
otherStatus = ['Private & Independent', 'Assets Acquired']

Map to Failed: 1, NotFailed: 0, and Unknown: -1

In [169]:
conditions = [
    html.ownershipstatus.isin(failedStatus),
    html.ownershipstatus.isin(exitedStatus),
    html.ownershipstatus.isin(otherStatus)
]
values = [1, 0, -1]
html['failed'] = np.select(conditions, values)

Define the the end year and start year

In [170]:
# Define end year of a company `end_yr`
html['end_yr'] = html['exit_date'].str.slice(start=0, stop=4) \
    .astype('float', errors='ignore') # This will have NAs

# Start yr `start_yr`
html['start_yr'] = html['startdate'].str.slice(start=0, stop=4) \
    .fillna(html['lastVC'].str.slice(start=0, stop=4)) \
    .astype('float', errors='ignore')

Many companies (80%) will not have an end year. We are estimating it as their lastVC date + 2 years.

Also, define company lifespan

In [171]:
html.end_yr = html.end_yr.fillna(html.lastVC.str.slice(start=0, stop=4).astype(int) + 2)

# Lifespan (end_yr - start_yr)
html['lifespan'] = html['end_yr'] - html['start_yr']

In [174]:
html.to_csv('../data/07_28_merged_analysis.csv', index=False)